![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_RoBertaForQuestionAnswering.ipynb)

## Import ONNX RoBertaForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `RoBertaForQuestionAnswering` is only available since in `Spark NLP 5.1.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import RoBERTa models trained/fine-tuned for question answering via `RobertaForQuestionAnswering` or `TFRobertaForQuestionAnswering`. These models are usually under `Question Answering` category and have `roberta` in their labels
- Reference: [TFRobertaForQuestionAnswering](https://huggingface.co/docs/transformers/model_doc/roberta#transformers.TFRobertaForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=roberta&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [3]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 22.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 116.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [deepset/roberta-base-squad2](https://huggingface.co/deepset/roberta-base-squad2) model from HuggingFace as an example as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.

In [4]:
from optimum.onnxruntime import ORTModelForQuestionAnswering
import tensorflow as tf

MODEL_NAME = 'deepset/roberta-base-squad2'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

(…)rta-base-squad2/resolve/main/config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Framework not specified. Using pt to export to ONNX.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

(…)quad2/resolve/main/tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

(…)erta-base-squad2/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)erta-base-squad2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)ad2/resolve/main/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [5]:
import json

# Read the vocab JSON file
with open('{}/vocab.json'.format(ONNX_MODEL), 'r') as json_file:
    tokenizer = json.load(json_file)

# let's save the vocab as txt file
with open('{}/vocab.txt'.format(ONNX_MODEL), 'w') as keys_file:
  for item in tokenizer.keys():
    keys_file.write("%s\n" % item)

Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {ONNX_MODEL}

total 488576
-rw-r--r-- 1 root root       749 Oct 17 16:47 config.json
-rw-r--r-- 1 root root    456318 Oct 17 16:47 merges.txt
-rw-r--r-- 1 root root 496507989 Oct 17 16:47 model.onnx
-rw-r--r-- 1 root root       957 Oct 17 16:47 special_tokens_map.json
-rw-r--r-- 1 root root      1394 Oct 17 16:47 tokenizer_config.json
-rw-r--r-- 1 root root   2108614 Oct 17 16:47 tokenizer.json
-rw-r--r-- 1 root root    798293 Oct 17 16:47 vocab.json
-rw-r--r-- 1 root root    407065 Oct 17 16:47 vocab.txt


- As you can see, we need to move `vocab.txt` and `merges.txt` from the tokenizer to `assets` folder which Spark NLP will look for

In [7]:
!mkdir {ONNX_MODEL}/assets

In [8]:
!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets

In [9]:
!mv {ONNX_MODEL}/merges.txt {ONNX_MODEL}/assets

Voila! We have our `vocab.txt` and `merges.txt` inside assets directory

In [10]:
!ls -lR {ONNX_MODEL}

onnx_models/deepset/roberta-base-squad2:
total 487732
drwxr-xr-x 2 root root      4096 Oct 17 16:47 assets
-rw-r--r-- 1 root root       749 Oct 17 16:47 config.json
-rw-r--r-- 1 root root 496507989 Oct 17 16:47 model.onnx
-rw-r--r-- 1 root root       957 Oct 17 16:47 special_tokens_map.json
-rw-r--r-- 1 root root      1394 Oct 17 16:47 tokenizer_config.json
-rw-r--r-- 1 root root   2108614 Oct 17 16:47 tokenizer.json
-rw-r--r-- 1 root root    798293 Oct 17 16:47 vocab.json

onnx_models/deepset/roberta-base-squad2/assets:
total 848
-rw-r--r-- 1 root root 456318 Oct 17 16:47 merges.txt
-rw-r--r-- 1 root root 407065 Oct 17 16:47 vocab.txt


## Import and Save RoBertaForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [11]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.1.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.5/537.5 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.9 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [12]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()
print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `RoBertaForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [13]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = RoBertaForQuestionAnswering.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [14]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [15]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your RoBertaForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [16]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 484956
drwxr-xr-x 4 root root      4096 Oct 17 16:49 fields
drwxr-xr-x 2 root root      4096 Oct 17 16:49 metadata
-rw-r--r-- 1 root root 496583922 Oct 17 16:49 roberta_classification_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [18]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = RoBertaForQuestionAnswering.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

context = """The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species."""
question = "Which name is also used to describe the Amazon rainforest in English?"
example = spark.createDataFrame([[question, context]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+---------------------------+
|result                     |
+---------------------------+
|[as Amazonia or the Amazon]|
+---------------------------+



That's it! You can now go wild and use hundreds of `RoBertaForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
